# Minor Project 

In [21]:
import pandas as pd
from data import *

In [22]:
data = pd.read_csv('Data/Student Mental Wellness Survey.csv')
data.columns = [f'X{i}' for i in range(1, len(data.columns) + 1)]
data = data.iloc[:, 1:]
data.head()

,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18
0,20,Bhubaneswar,Male,Bachelor's 3rd Year,Computer Science,Everyday,Often,Weight gain,Often,Everyday,Everyday,Everyday,Rarely,No change,Nothing,Yes,No
1,69,Liquid,Male,Bachelor's 3rd Year,Computer Science,Everyday,Everyday,No change,Everyday,Everyday,Sometimes,Everyday,Everyday,No change,nope,Yes,Yes
2,20,Faridabad,Male,Bachelor's 3rd Year,BBA,Sometimes,Rarely,No change,No,No,No,No,No,No change,No,No,Yes
3,21,UP,Male,Bachelor's 4th Year,CSE,No,No,No change,No,No,No,No,No,No change,NaN,Yes,Yes
4,56,Delhi,Female,Bachelor's 2nd Year,BA,Rarely,Rarely,Increased appetite,Rarely,Rarely,Rarely,Rarely,Rarely,Slight decline,NaN,No,No


In [23]:
data.loc[:, 'X3'] = data['X3'].str.strip().str.lower()
data.loc[:, 'X3'] = data['X3'].replace(replacements)
data.loc[:, 'X3'] = data['X3'].apply(lambda x: x.title())
data['X3'] = data['X3'].apply(lambda x: x if x in indian_states_ut else 'Others')
data['X3'].value_counts()

X3
Odisha            35
Maharashtra       31
Madhya Pradesh    19
Jharkhand         18
Bihar             18
Uttar Pradesh     14
Assam              7
West Bengal        6
Delhi              6
Haryana            6
Others             5
Chhattisgarh       4
Karnataka          3
Rajasthan          2
Tamil Nadu         1
Gujarat            1
Goa                1
Name: count, dtype: int64

These columns only take 2-5 values hence one hot encoding is done

In [24]:
columns_to_encode = ['X3','X4','X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15']
data= pd.get_dummies(data, columns=columns_to_encode)
data.head()

,X2,X5,X6,X16,X17,X18,X3_Assam,X3_Bihar,X3_Chhattisgarh,X3_Delhi,...,X13_Rarely,X13_Sometimes,X14_Everyday,X14_No,X14_Often,X14_Rarely,X14_Sometimes,X15_No change,X15_Significant decline,X15_Slight decline
0,20,Bachelor's 3rd Year,Computer Science,Nothing,Yes,No,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False
1,69,Bachelor's 3rd Year,Computer Science,nope,Yes,Yes,False,False,False,False,...,False,False,True,False,False,False,False,True,False,False
2,20,Bachelor's 3rd Year,BBA,No,No,Yes,False,False,False,False,...,False,False,False,True,False,False,False,True,False,False
3,21,Bachelor's 4th Year,CSE,NaN,Yes,Yes,False,False,False,False,...,False,False,False,True,False,False,False,True,False,False
4,56,Bachelor's 2nd Year,BA,NaN,No,No,False,False,False,True,...,True,False,False,False,False,True,False,False,False,True


These columns can take more than one value so only the top 10 are taken

In [25]:
data['X6'] = data['X6'].str.strip()
data['X6'] = data['X6'].map(category_mapping)

In [26]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer, PorterStemmer

data['X6'].value_counts()
data['X16'] = data['X16'].astype(str)

sia = SentimentIntensityAnalyzer()
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

negative_responses = ['no', 'nope', 'nothing', 'na','nope.','nah','not at all','no.', ]

data['X16_sentiment'] = data['X16'].apply(lambda x: 0.0 if any(token in [stemmer.stem(lemmatizer.lemmatize(word)) for word in nltk.word_tokenize(x.lower())] for token in negative_responses)   else sia.polarity_scores(x)['compound'] if isinstance(x, str) else 0.0)
data


,X2,X5,X6,X16,X17,X18,X3_Assam,X3_Bihar,X3_Chhattisgarh,X3_Delhi,...,X13_Sometimes,X14_Everyday,X14_No,X14_Often,X14_Rarely,X14_Sometimes,X15_No change,X15_Significant decline,X15_Slight decline,X16_sentiment
0,20,Bachelor's 3rd Year,Computer Science,Nothing,Yes,No,False,False,False,False,...,False,False,False,False,True,False,True,False,False,0.0000
1,69,Bachelor's 3rd Year,Computer Science,nope,Yes,Yes,False,False,False,False,...,False,True,False,False,False,False,True,False,False,0.0000
2,20,Bachelor's 3rd Year,Others,No,No,Yes,False,False,False,False,...,False,False,True,False,False,False,True,False,False,0.0000
3,21,Bachelor's 4th Year,Computer Science,nan,Yes,Yes,False,False,False,False,...,False,False,True,False,False,False,True,False,False,0.0000
4,56,Bachelor's 2nd Year,Others,nan,No,No,False,False,False,True,...,False,False,False,False,True,False,False,False,True,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,22,Bachelor's 4th Year,Computer Science,.,No,No,False,False,False,False,...,False,False,True,False,False,False,False,False,True,0.0000
173,24,Working,Others,No,No,No,False,False,False,False,...,False,False,True,False,False,False,True,False,False,0.0000
174,21,Bachelor's 4th Year,Others,nan,Yes,Yes,False,False,False,False,...,False,False,False,False,False,True,False,False,True,0.0000
175,20,Bachelor's 3rd Year,Computer Science,Tired of college and lack of holidays/outings.,I am not sure.,No,False,False,False,False,...,False,False,False,False,True,False,False,True,False,-0.6369


In [27]:
top_categories = 10
columns_to_encode = [ 'X5', 'X6']
encoded_columns = pd.DataFrame()
for column in columns_to_encode:
    top_categories_series = data[column].value_counts().nlargest(top_categories)
    top_categories_list = top_categories_series.index.tolist()
    
    encoded = pd.get_dummies(data[column].apply(lambda x: x if x in top_categories_list else 'Other'), prefix=column)
    encoded_columns = pd.concat([encoded_columns, encoded], axis=1)
data_encoded = pd.concat([data.drop(columns=columns_to_encode), encoded_columns], axis=1)
data_encoded.head()

,X2,X16,X17,X18,X3_Assam,X3_Bihar,X3_Chhattisgarh,X3_Delhi,X3_Goa,X3_Gujarat,...,X6_BSc,X6_Commerce,X6_Computer Science,X6_Design,X6_Engineering,X6_Fashion,X6_MBA,X6_Medical,X6_Others,X6_Psychology
0,20,Nothing,Yes,No,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
1,69,nope,Yes,Yes,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,20,No,No,Yes,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,21,nan,Yes,Yes,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
4,56,nan,No,No,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False


In [28]:
def merge_columns(row):
    if row['X18'] == 'Yes':
        return True
    elif row['X17'] == 'Yes':
        return True
    else: 
        return False

data_encoded['Output'] = data_encoded.apply(merge_columns, axis=1)
data_encoded.drop(['X18', 'X17'], axis=1, inplace=True)

data_encoded.head()

,X2,X16,X3_Assam,X3_Bihar,X3_Chhattisgarh,X3_Delhi,X3_Goa,X3_Gujarat,X3_Haryana,X3_Jharkhand,...,X6_Commerce,X6_Computer Science,X6_Design,X6_Engineering,X6_Fashion,X6_MBA,X6_Medical,X6_Others,X6_Psychology,Output
0,20,Nothing,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
1,69,nope,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
2,20,No,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
3,21,nan,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
4,56,nan,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [29]:
data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 89 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   X2                         177 non-null    int64  
 1   X16                        177 non-null    object 
 2   X3_Assam                   177 non-null    bool   
 3   X3_Bihar                   177 non-null    bool   
 4   X3_Chhattisgarh            177 non-null    bool   
 5   X3_Delhi                   177 non-null    bool   
 6   X3_Goa                     177 non-null    bool   
 7   X3_Gujarat                 177 non-null    bool   
 8   X3_Haryana                 177 non-null    bool   
 9   X3_Jharkhand               177 non-null    bool   
 10  X3_Karnataka               177 non-null    bool   
 11  X3_Madhya Pradesh          177 non-null    bool   
 12  X3_Maharashtra             177 non-null    bool   
 13  X3_Odisha                  177 non-null    bool   

In [30]:
data_encoded.to_csv('Data/encoded_data.csv', index=False)